In [65]:
import pandas as pd
import re
import numpy as np
from scipy import spatial

In [2]:
df = pd.read_csv('./processed_product_data.csv')

In [3]:
cols_to_use = ['group', 'subgroup', 'variety', 'grade', 'color', 'length', 'boxtype']

In [4]:
df = df[cols_to_use]

In [5]:
df['color'] = df['color'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', '', x))

In [6]:
import requests
import json

In [7]:
post_data = {
    "limit": 2000,
#     "filters": [{
#         "field": "product.grade.keyword",
#         "value": "6 Stem"
#     }, {
#         "field": "locationAddress.country.keyword",
#         "value": "United States"
#     }]
}

In [8]:
r = requests.post("http://13.126.59.19:20021/api/search/VendorProducts",data=json.dumps(post_data))

In [9]:
print(r.status_code, r.reason)

200 OK


In [10]:
result = json.loads(r.text)

In [11]:
df = pd.io.json.json_normalize(result['products'])

In [12]:
df.iloc[0]

locationAddress.addressLine1                                     High Street Bogota
locationAddress.city                                                         Bogota
locationAddress.country                                                    Columbia
locationAddress.postalCode                                                    12345
locationAddress.state                                                        Bogota
locationName                                                       Exotic Farms SAS
product.boxType                                                    J Type Full Box 
product.color                                                            Lilac/Pink
product.grade                                                                Alesso
product.group                                                              Supplies
product.id                                     e1982adc-d482-4b17-b1b1-f1a75e720d0a
product.images                    [https://s3.ap-south-1.amazonaws.com/aob-d

In [13]:
import spacy

In [14]:
nlp = spacy.load('en_core_web_lg')

In [15]:
def get_word_vectors(words):
    # converts a list of words into their word vectors
    return [nlp(str(word)).vector for word in words]

In [16]:
columns_to_use = ['locationAddress.city', 'locationAddress.state', 'locationAddress.country', 'locationName',
                  'product.color', 'product.grade', 'product.group', 'product.variety', 'productOffer.boxType',
                  'productOffer.height', 'productOffer.length', 'productOffer.price.amount',
                  'productOffer.quantityPerBox', 'vendorName']

In [17]:
data_df = df[columns_to_use]

In [18]:
def tokenize_on_caps(x):
    x = ' '.join(re.findall('[A-Z,(1-9)][^A-Z]*', x))
    return re.sub('[^A-Za-z0-9 ]+', '', x.strip())

In [19]:
data_df['product.color'] = data_df['product.color'].apply(lambda x: tokenize_on_caps(x))

/home/ganesh/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
data_df['productOffer.price.amount'] = data_df['productOffer.price.amount'].apply(lambda x: round(x,2))

/home/ganesh/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
new_df = data_df[columns_to_use].apply(get_word_vectors)

In [22]:
count = 0
for i in new_df.columns:
    for j in range(len(new_df)):
        sum_ = sum(new_df.loc[j, i])
        if sum_ == 0:
            count += 1
            print("index: {0}, column: {1}, value: {2}\n".format(j, i, data_df.loc[j,i]))

index: 5, column: product.color, value: Varigated

index: 138, column: product.color, value: Bicolor

index: 142, column: product.color, value: Varigated

index: 270, column: product.color, value: Nobbio

index: 335, column: product.color, value: Bicolor

index: 559, column: product.color, value: Bicolor

index: 569, column: product.color, value: Bicolor

index: 696, column: product.color, value: Nobbio

index: 745, column: product.color, value: Varigated

index: 752, column: product.color, value: Varigated

index: 932, column: product.color, value: Nobbio

index: 1009, column: product.color, value: Bicolor

index: 1039, column: product.color, value: Bicolor

index: 1086, column: product.color, value: Nobbio

index: 1422, column: product.color, value: Nobbio

index: 1471, column: product.color, value: Bicolor

index: 1503, column: product.color, value: Nobbio

index: 1544, column: product.color, value: Varigated

index: 1555, column: product.color, value: Nobbio

index: 1588, column: p

In [23]:
count

364

In [26]:
sum([np.abs(i) for i in (nlp('Exotic Farms SAS').vector - nlp('Exotic Farms').vector)])

41.24900347460061

In [27]:
from scipy import spatial

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
df.values[0][0]

'High Street Bogota'

In [64]:
new_df

,locationAddress.city,locationAddress.state,locationAddress.country,locationName,product.color,product.grade,product.group,product.variety,productOffer.boxType,productOffer.height,productOffer.length,productOffer.price.amount,productOffer.quantityPerBox,vendorName
0,"[0.55778, 0.17659, 0.45883, -0.71324, 0.63613,...","[0.55778, 0.17659, 0.45883, -0.71324, 0.63613,...","[0.10914, -0.11436, 0.10107, -0.81965, 0.4281,...","[0.010611673, 0.21350335, 0.20271367, -0.34960...","[0.16487199, 0.033315003, -0.21367499, -0.2181...","[0.58458, 0.30535, 0.19941, 1.1952, -0.29308, ...","[-0.20812, 0.2152, 0.38998, -0.22682, -0.0325,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.17609, 0.55718, -0.11365, 0.32519, 0.28829,...","[-0.62684, 0.72843, 0.47222, -0.083358, 0.0913...","[-0.62684, 0.72843, 0.47222, -0.083358, 0.0913...","[-0.26355, 0.48304, -0.16475, 0.38882, 0.4374,...","[0.2488, 0.2567, 0.20715, -0.072082, 0.19968, ...","[-0.1535, 0.147745, -0.496825, -0.463865, 0.17..."
1,"[0.55778, 0.17659, 0.45883, -0.71324, 0.63613,...","[0.55778, 0.17659, 0.45883, -0.71324, 0.63613,...","[0.10914, -0.11436, 0.10107, -0.81965, 0.4281,...","[0.010611673, 0.21350335, 0.20271367, -0.34960...","[-0.038646, 0.16997, 0.094474, -0.066617, 0.10...","[-0.36357, 0.619865, 0.10990998, -0.2083525, 0...","[-0.21741125, 0.3714075, -0.150089, 0.03921251...","[0.61645496, 0.524135, -0.072501004, 0.0137650...","[0.17609, 0.55718, -0.11365, 0.32519, 0.28829,...","[-0.62684, 0.72843, 0.47222, -0.083358, 0.0913...","[-0.62684, 0.72843, 0.47222, -0.083358, 0.0913...","[-0.24235, 0.57122, 0.088702, 0.32429, 0.06199...","[0.18269, 0.45622, 0.14485, -0.14701, -0.00058...","[-0.1535, 0.147745, -0.496825, -0.463865, 0.17..."
2,"[-0.062242, 0.23847, 0.45944, -0.2806, 0.88573...","[-0.27047, 0.80905, 0.088226, -0.69639, 0.8688...","[-0.1438605, -0.0200435, 0.34185, -0.402145, 0...","[0.0502, 0.47306, 0.065879, -0.43719, 0.224220...","[0.096765004, 0.026519999, -0.299513, -0.19842...","[-0.44219, 0.73456, 0.08908751, 0.06682775, 0....","[-0.25901002, 0.57634, 0.320645, -0.46744, 0.1...","[0.113250904, 0.204443, -0.249724, -0.01601325...","[0.17609, 0.55718, -0.11365, 0.32519, 0.28829,...","[-0.62684, 0.72843, 0.47222, -0.083358, 0.0913...","[-0.62684, 0.72843, 0.47222, -0.083358, 0.0913...","[-0.65264, 0.58544, -0.073324, 0.39464, 0.2108...","[-0.53441, 0.55667, -0.11892, 0.21249, 0.04592...","[0.0502, 0.47306, 0.065879, -0.43719, 0.224220..."
3,"[0.55778, 0.17659, 0.45883, -0.71324, 0.63613,...","[0.55778, 0.17659, 0.45883, -0.71324, 0.63613,...","[0.10914, -0.11436, 0.10107, -0.81965, 0.4281,...","[0.27889, 0.088295, 0.229415, -0.35662, 0.3180...","[0.068227, 0.2252, -0.07203, 0.28968, -0.01310...","[-0.123245, 0.69401, 0.520595, 0.228541, -0.08...","[-0.1659245, 0.28003, -0.4763895, -0.40396002,...","[-0.063603334, -0.09333333, 0.18202834, 0.1090...","[0.17609, 0.55718, -0.11365, 0.32519, 0.28829,...","[-0.62684, 0.72843, 0.47222, -0.083358, 0.0913...","[-0.62684, 0.72843, 0.47222, -0.083358, 0.0913...","[-0.65264, 0.58544, -0.073324, 0.39464, 0.2108...","[-0.53441, 0.55667, -0.11892, 0.21249, 0.04592...","[-0.244615, 0.25001, -0.046916, -0.316025, 0.1..."
4,"[0.55778, 0.17659, 0.45883, -0.71324, 0.63613,...","[0.55778, 0.17659, 0.45883, -0.71324, 0.63613,...","[0.10914, -0.11436, 0.10107, -0.81965, 0.4281,...","[0.010611673, 0.21350335, 0.20271367, -0.34960...","[0.014344998, -0.075500995, -0.010384001, -0.2...","[0.05601999, -0.3235515, -0.14327002, -0.27015...","[-0.63514, 0.0048199967, -0.21467151, 0.216815...","[0.42442, -0.18025, 0.24931, 0.29084, 0.16996,...","[0.17609, 0.55718, -0.11365, 0.32519, 0.28829,...","[-0.62684, 0.72843, 0.47222, -0.083358, 0.0913...","[-0.62684, 0.72843, 0.47222, -0.083358, 0.0913...","[-0.26355, 0.48304, -0.16475, 0.38882, 0.4374,...","[-0.53441, 0.55667, -0.11892, 0.21249, 0.04592...","[-0.1535, 0.147745, -0.496825, -0.463865, 0.17..."
5,"[-0.062242, 0.23847, 0.45944, -0.2806, 0.88573...","[-0.27047, 0.80905, 0.088226, 

In [56]:
data_df.iloc[0][0]

'Bogota'

In [57]:
data_df.iloc[4][4]

'Purple Blue'

In [73]:
1 - spatial.distance.cosine(new_df.iloc[3][1], new_df.iloc[4][3])

0.11136337369680405

In [72]:
data_df.iloc[3][1], data_df.iloc[4][3]

('Bogota', 'Exotic Farms SAS')

In [99]:
feature_weights = {'locationAddress.city':1, 'locationAddress.state':1,
                   'locationAddress.country':1, 'locationName':1, 'product.color':2,
                   'product.grade':1, 'product.group':1, 'product.variety':2,
                   'productOffer.boxType':1, 'productOffer.height':1, 'productOffer.length':1,
                   'productOffer.price.amount':1, 'productOffer.quantityPerBox':1, 'vendorName':1}

In [100]:
def get_similarity(u, v):
    return 1 - spatial.distance.cosine(u, v)

In [103]:
data_df

,locationAddress.city,locationAddress.state,locationAddress.country,locationName,product.color,product.grade,product.group,product.variety,productOffer.boxType,productOffer.height,productOffer.length,productOffer.price.amount,productOffer.quantityPerBox,vendorName
0,Bogota,Bogota,Columbia,Exotic Farms SAS,Lilac Pink,Alesso,Supplies,Solfarino,FB,10,10,0.39,400,Tulipan Flowers
1,Bogota,Bogota,Columbia,Exotic Farms SAS,Fall,8-11 Bloom,Pre-made bouquets,Fifth Avenue,FB,10,10,0.12,300,Tulipan Flowers
2,Miami,Florida,United States,Polo Farms,Lavender Purple,8-10 cm,Cut Greens,With All My Heart,FB,10,10,0.95,100,Polo Farms
3,Bogota,Bogota,Columbia,Mounika Bogota,Antique,18 Gram,Fresh flowers,Red Velvet Henriette,FB,10,10,0.95,100,Mounika Farms
4,Bogota,Bogota,Columbia,Exotic Farms SAS,Purple Blue,Monte Lava,Collateral materials,Catwalk,FB,10,10,0.39,100,Tulipan Flowers
5,Miami,Florida,United States,Sande Miami,Varigated,10 St,Plants,Verena,FB,10,10,0.38,200,Calla Flowers By Sande
6,Miami,Holland,Netherlands,Premium Dutch Mia,Light Green,Extra,Collateral materials,Takiny,FB,10,10,0.18,100,Tuning
7,Miami,Holland,Netherlands,Premium Dutch Mia,Green Yellow,6.5 Inch,Plants,Amina,FB,10,10,0.95,200,Tuning
8,Quito,Quito,Ecuador,Bellaflor Group-Special,Green Antique,Titanium,Plants,Preziosa,FB,10,10,0.79,100,Exportadora de Flores
9,Miami,Florida,United States,Polo Farms,Solid Purple,Chico,Fresh flowers,Carousel,FB,10,10,0.72,100,Polo Farms


In [126]:
max_sim = sum(feature_weights.values())

In [114]:
lower_triangular_matrix = np.zeros((data_df.shape[0], data_df.shape[0]))
for i in range(new_df.shape[0]):
    for j in range(new_df.shape[0]):
        if i==j:
            lower_triangular_matrix[i][j] = max_sim/2
            break
        try:
            lower_triangular_matrix[i][j] = sum([get_similarity(new_df.loc[i,key], new_df.loc[j,key]) * feature_weights[key] 
                                                 if not np.isnan(get_similarity(new_df.loc[i,key], new_df.loc[j,key])) else 0 
                                                 for key in data_df.columns])
        except:
            print(nlp(data_df.iloc[i][j]), nlp(data_df.iloc[j][i]))
            break

In [128]:
similarity_mat = lower_triangular_matrix + lower_triangular_matrix.T

In [129]:
np.save('similarity_matrix.npy', similarity_mat)

In [130]:
similarity_mat

array([[ 2.        , 10.58671823,  8.20704451, ...,  8.34617777,
         8.82449529,  9.54275803],
       [10.58671823,  2.        ,  8.8427408 , ...,  7.792638  ,
         7.98891318,  8.29967605],
       [ 8.20704451,  8.8427408 ,  2.        , ..., 11.31455928,
         8.35312512, 10.54209743],
       ...,
       [ 8.34617777,  7.792638  , 11.31455928, ...,  2.        ,
         7.82390344, 10.26675783],
       [ 8.82449529,  7.98891318,  8.35312512, ...,  7.82390344,
         2.        ,  8.8683397 ],
       [ 9.54275803,  8.29967605, 10.54209743, ..., 10.26675783,
         8.8683397 ,  2.        ]])

In [158]:
sims = np.argsort(similarity_mat[0])[-5:]

In [148]:
similarity_mat[0][92]

12.443401962518692

In [145]:
similarity_mat[0][186]

12.674015700817108

In [156]:
similarity_mat[0][1795]

12.526574492454529

In [164]:
def get_similars(df, indices):
    for index in reversed(indices):
        print(df.iloc[index].values)

In [169]:
data_df.iloc[0].values

array(['Bogota', 'Bogota', 'Columbia', 'Exotic Farms SAS', 'Lilac Pink',
       'Alesso', 'Supplies', 'Solfarino', 'FB', 10, 10, 0.39, 400,
       'Tulipan Flowers'], dtype=object)

In [167]:
get_similars(data_df, sims)

['Bogota' 'Bogota' 'Columbia' 'Exotic Farms SAS' 'Pink Blue' 'Mega'
 'Supplies' 'Ecroland' 'FB' 10 10 0.72 400 'Tulipan Flowers']
['Bogota' 'Bogota' 'Columbia' 'Exotic Farms SAS' 'White Dyed Red'
 'Ecuadorian' 'Supplies' 'Prince Of Darkness' 'FB' 10 10 0.18 300
 'Tulipan Flowers']
['Bogota' 'Bogota' 'Columbia' 'Exotic Farms SAS' 'Pink Purple' 'Mega'
 'Supplies' 'Chapeau' 'FB' 10 10 0.39 100 'Tulipan Flowers']
['Bogota' 'Bogota' 'Columbia' 'Exotic Farms SAS' 'Light Purple' 'Chico'
 'Supplies' 'Lemoncello' 'FB' 10 10 0.65 300 'Tulipan Flowers']
['Bogota' 'Bogota' 'Columbia' 'Exotic Farms SAS' 'Brown Orange' 'Alesso'
 'Collateral materials' 'A1' 'FB' 10 10 0.79 100 'Tulipan Flowers']
